## Feature Engineering

In [167]:
import pandas as pd
import os
import nltk
import string
import datetime as dt
from datetime import datetime, timedelta

nltk.download()
string.punctuation

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [124]:
# Set path to cryptocurrency csv files
path = '../Data/Reddit_Comments/Cryptocurrency/'

files = os.listdir(path)

# Create list of csv file names
csv_list = []
for f in files:
    csv_list.append(f)

In [125]:
csv_list = csv_list[:-1]
csv_list

['Cryptocurrency_2021-08-07.csv',
 'Cryptocurrency_2021-08-08.csv',
 'Cryptocurrency_2021-08-09.csv',
 'Cryptocurrency_2021-08-10.csv',
 'Cryptocurrency_2021-08-11.csv',
 'Cryptocurrency_2021-08-12.csv',
 'Cryptocurrency_2021-08-13.csv',
 'Cryptocurrency_2021-08-14.csv',
 'Cryptocurrency_2021-08-15.csv',
 'Cryptocurrency_2021-08-16.csv',
 'Cryptocurrency_2021-08-17.csv',
 'Cryptocurrency_2021-08-18.csv',
 'Cryptocurrency_2021-08-19.csv',
 'Cryptocurrency_2021-08-20.csv',
 'Cryptocurrency_2021-08-21.csv',
 'Cryptocurrency_2021-08-22.csv',
 'Cryptocurrency_2021-08-23.csv',
 'Cryptocurrency_2021-08-24.csv',
 'Cryptocurrency_2021-08-25.csv',
 'Cryptocurrency_2021-08-26.csv',
 'Cryptocurrency_2021-08-27.csv',
 'Cryptocurrency_2021-08-28.csv',
 'Cryptocurrency_2021-08-29.csv',
 'Cryptocurrency_2021-08-30.csv',
 'Cryptocurrency_2021-08-31.csv',
 'Cryptocurrency_2021-09-01.csv',
 'Cryptocurrency_2021-09-02.csv',
 'Cryptocurrency_2021-09-03.csv',
 'Cryptocurrency_2021-09-04.csv',
 'Cryptocurren

In [126]:
csv_list[0]

'Cryptocurrency_2021-08-07.csv'

In [127]:
df = pd.read_csv(path + csv_list[0])
df.head()

,author,author_fullname,author_premium,body,collapsed_reason_code,comment_type,created_utc,score,id,parent_id,permalink
0,HiCarumba,t2_95jx76vi,False,Yes Brother. This is the way.,NaN,NaN,1628333279,1,h81jxny,t1_h81jvaa,/r/CryptoCurrency/comments/ozr28z/it_was_prophesied/h81jxny/
1,baeiby,t2_7htt0be3,False,"Lol to be very honest, I knew close to zilch abt investing in general. Didn't even know what DCA...",NaN,NaN,1628333279,2,h81jxnr,t1_h81jn5u,/r/CryptoCurrency/comments/ozqz9m/i_trust_people_here_more_than_my_circle_of/h81jxnr/
2,NotRyanPace,t2_xgk0k,False,"End of year, take the ""Last call for ETH under $10k"" comments seriously. That's a reasonable pri...",NaN,NaN,1628333278,1,h81jxmq,t1_h81jqki,/r/CryptoCurrency/comments/ozinnf/daily_discussion_august_7_2021_gmt0/h81jxmq/
3,BountyManagerBD,t2_8nbdzghw,False,ESCROW PROTOCOL are aiming to attract users from outside of the cryptocurrency space,NaN,NaN,1628333277,1,h81jxlg,t3_o2e67b,/r/CryptoCurrency/comments/o2e67b/escrow_protocol_ico_crowdfunding_platform_with/h81jxlg/
4,Gabus_Bego,t2_bbs0wmdt,False,"Thank you, my dude.",NaN,NaN,1628333276,1,h81jxk6,t1_h81jtfb,/r/CryptoCurrency/comments/ozinnf/daily_discussion_august_7_2021_gmt0/h81jxk6/


In [128]:
# Remove all rows that have NaN in df['body']
df = df[df['body'].notna()]

,author,author_fullname,author_premium,body,collapsed_reason_code,comment_type,created_utc,score,id,parent_id,permalink
0,HiCarumba,t2_95jx76vi,False,Yes Brother. This is the way.,NaN,NaN,1628333279,1,h81jxny,t1_h81jvaa,/r/CryptoCurrency/comments/ozr28z/it_was_prophesied/h81jxny/
1,baeiby,t2_7htt0be3,False,"Lol to be very honest, I knew close to zilch abt investing in general. Didn't even know what DCA...",NaN,NaN,1628333279,2,h81jxnr,t1_h81jn5u,/r/CryptoCurrency/comments/ozqz9m/i_trust_people_here_more_than_my_circle_of/h81jxnr/
2,NotRyanPace,t2_xgk0k,False,"End of year, take the ""Last call for ETH under $10k"" comments seriously. That's a reasonable pri...",NaN,NaN,1628333278,1,h81jxmq,t1_h81jqki,/r/CryptoCurrency/comments/ozinnf/daily_discussion_august_7_2021_gmt0/h81jxmq/
3,BountyManagerBD,t2_8nbdzghw,False,ESCROW PROTOCOL are aiming to attract users from outside of the cryptocurrency space,NaN,NaN,1628333277,1,h81jxlg,t3_o2e67b,/r/CryptoCurrency/comments/o2e67b/escrow_protocol_ico_crowdfunding_platform_with/h81jxlg/
4,Gabus_Bego,t2_bbs0wmdt,False,"Thank you, my dude.",NaN,NaN,1628333276,1,h81jxk6,t1_h81jtfb,/r/CryptoCurrency/comments/ozinnf/daily_discussion_august_7_2021_gmt0/h81jxk6/


In [129]:
df.isna().any()

author                   False
author_fullname           True
author_premium            True
body                     False
collapsed_reason_code     True
comment_type              True
created_utc              False
score                    False
id                       False
parent_id                False
permalink                False
dtype: bool

In [130]:
# Remove automoderator authors
df = df[df.author != "AutoModerator"]

,author,author_fullname,author_premium,body,collapsed_reason_code,comment_type,created_utc,score,id,parent_id,permalink
0,HiCarumba,t2_95jx76vi,False,Yes Brother. This is the way.,NaN,NaN,1628333279,1,h81jxny,t1_h81jvaa,/r/CryptoCurrency/comments/ozr28z/it_was_prophesied/h81jxny/
1,baeiby,t2_7htt0be3,False,"Lol to be very honest, I knew close to zilch abt investing in general. Didn't even know what DCA...",NaN,NaN,1628333279,2,h81jxnr,t1_h81jn5u,/r/CryptoCurrency/comments/ozqz9m/i_trust_people_here_more_than_my_circle_of/h81jxnr/
2,NotRyanPace,t2_xgk0k,False,"End of year, take the ""Last call for ETH under $10k"" comments seriously. That's a reasonable pri...",NaN,NaN,1628333278,1,h81jxmq,t1_h81jqki,/r/CryptoCurrency/comments/ozinnf/daily_discussion_august_7_2021_gmt0/h81jxmq/
3,BountyManagerBD,t2_8nbdzghw,False,ESCROW PROTOCOL are aiming to attract users from outside of the cryptocurrency space,NaN,NaN,1628333277,1,h81jxlg,t3_o2e67b,/r/CryptoCurrency/comments/o2e67b/escrow_protocol_ico_crowdfunding_platform_with/h81jxlg/
4,Gabus_Bego,t2_bbs0wmdt,False,"Thank you, my dude.",NaN,NaN,1628333276,1,h81jxk6,t1_h81jtfb,/r/CryptoCurrency/comments/ozinnf/daily_discussion_august_7_2021_gmt0/h81jxk6/


In [131]:
comments_df = df.copy()

In [133]:
bitcoin_df = df[df['body'].str.contains('Bitcoin|BTC', case=False)]

,author,author_fullname,author_premium,body,collapsed_reason_code,comment_type,created_utc,score,id,parent_id,permalink
6,atomwest314,t2_x50okt,True,i noticed this too peeps like 'o wow ICP go up must be good now??' like nope... btc and eth risi...,NaN,NaN,1628333275,3,h81jxhz,t3_ozr2hx,/r/CryptoCurrency/comments/ozr2hx/can_we_please_remember_that_your_coin_being_in/h81jxhz/
23,Shrappy16,t2_4e2hp2d0,False,"Probably know the answer: If you earned enough in ETH to buy 1 BTC, would you do it or keep rid...",NaN,NaN,1628333265,1,h81jx4f,t3_oznwv4,/r/CryptoCurrency/comments/oznwv4/eth_just_completed_a_historymaking_16_green/h81jx4f/
29,Odd_Copy_8077,t2_ajd0894j,False,"A boy asked his bitcoin-investing dad for 1 bitcoin for his birthday\n\nDad: What? 40,660 dollar...",NaN,NaN,1628333261,4,h81jwxn,t1_h81jr69,/r/CryptoCurrency/comments/ozinnf/daily_discussion_august_7_2021_gmt0/h81jwxn/
91,Much-Weekend-8882,t2_66sa37r1,False,Is this all time high of eth/btc ratio?,NaN,NaN,1628333204,2,h81juiz,t3_ozinnf,/r/CryptoCurrency/comments/ozinnf/daily_discussion_august_7_2021_gmt0/h81juiz/
114,itsglocx,t2_c3slvp4i,False,can you imagine being mad at bitcoin going down but owning it at that price haha,NaN,NaN,1628341913,1,h81wcat,t3_ozsz9f,/r/CryptoCurrency/comments/ozsz9f/a_snapshot_in_time_back_when_bitcoin_holders/h81wcat/


## Sentiment Analysis with Vader

In [140]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('bmh')
import seaborn as sns
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [141]:
analyser = SentimentIntensityAnalyzer()

In [142]:
# function to calculate polarity scores
pol = lambda x: analyser.polarity_scores(x)

In [143]:
# creating new column 'polarity' in clean_df
comments_df['polarity'] = comments_df['body'].apply(pol)

In [144]:
# dropping unessential columns and seperating out 'polarity' dictionary
#df3 = pd.concat([clean_df.drop(['author_fullname'], axis=1), clean_df['polarity'].apply(pd.Series)], axis=1)
comments_df = pd.concat([comments_df, comments_df['polarity'].apply(pd.Series)], axis=1)

In [145]:
# print average polarity score for each label
print('The average Positive score = ' + str(comments_df['pos'].mean()))
print('The average Neutral score = ' + str(comments_df['neu'].mean()))
print('The average Negative score = ' + str(comments_df['neg'].mean()))
print('The average Compound score = ' + str(comments_df['compound'].mean()))

The average Positive score = 0.15040446479215724
The average Neutral score = 0.7915952921157251
The average Negative score = 0.05800017219026043
The average Compound score = 0.14927262883882847


In [93]:
#df['body'] = df['body'].astype(str)

## Data Cleaning

## Create new dataframe with average polarity scores for each day

In [178]:
d = dt.datetime.fromtimestamp(int(comments_df['created_utc'].iloc[0]))

In [179]:
str(d).split(' ', 1)[0]

'2021-08-07'

In [183]:
# new dataframe with average polarity score for each label
scores_df = pd.DataFrame([str(d).split(' ', 1)[0]], columns=['time'])
scores_df['avg_positive'] = str(comments_df['pos'].mean())
scores_df['avg_neutral'] = str(comments_df['neu'].mean())
scores_df['avg_negative'] = str(comments_df['neg'].mean())
scores_df['avg_compound'] = str(comments_df['compound'].mean())

In [184]:
scores_df.head()

,time,avg_positive,avg_neutral,avg_negative,avg_compound
0,2021-08-07,0.15040446479215724,0.7915952921157251,0.05800017219026043,0.14927262883882847


In [185]:
scores_df2 = pd.DataFrame(columns = ['time', 'avg_positive', 'avg_neutral', 'avg_negative', 'avg_compound'])

In [189]:
scores_df2.loc[0] = [str(d).split(' ', 1)[0], str(comments_df['pos'].mean()), str(comments_df['neu'].mean()), str(comments_df['neg'].mean()),
                    str(comments_df['compound'].mean())]

In [190]:
scores_df2.head()

,time,avg_positive,avg_neutral,avg_negative,avg_compound
0,2021-08-07,0.15040446479215724,0.7915952921157251,0.05800017219026043,0.14927262883882847


In [134]:
# Remove punctuation
def remove_punct(text):
    text_nopunct = "".join([char for char in text if char not in string.punctuation])# It will discard all punctuations
    return text_nopunct

df['body_text_clean'] = df['body'].apply(lambda x: remove_punct(x))

df.head()

,author,author_fullname,author_premium,body,collapsed_reason_code,comment_type,created_utc,score,id,parent_id,permalink,body_text_clean
0,HiCarumba,t2_95jx76vi,False,Yes Brother. This is the way.,NaN,NaN,1628333279,1,h81jxny,t1_h81jvaa,/r/CryptoCurrency/comments/ozr28z/it_was_prophesied/h81jxny/,Yes Brother This is the way
1,baeiby,t2_7htt0be3,False,"Lol to be very honest, I knew close to zilch abt investing in general. Didn't even know what DCA...",NaN,NaN,1628333279,2,h81jxnr,t1_h81jn5u,/r/CryptoCurrency/comments/ozqz9m/i_trust_people_here_more_than_my_circle_of/h81jxnr/,Lol to be very honest I knew close to zilch abt investing in general Didnt even know what DCA me...
2,NotRyanPace,t2_xgk0k,False,"End of year, take the ""Last call for ETH under $10k"" comments seriously. That's a reasonable pri...",NaN,NaN,1628333278,1,h81jxmq,t1_h81jqki,/r/CryptoCurrency/comments/ozinnf/daily_discussion_august_7_2021_gmt0/h81jxmq/,End of year take the Last call for ETH under 10k comments seriously Thats a reasonable price EOY
3,BountyManagerBD,t2_8nbdzghw,False,ESCROW PROTOCOL are aiming to attract users from outside of the cryptocurrency space,NaN,NaN,1628333277,1,h81jxlg,t3_o2e67b,/r/CryptoCurrency/comments/o2e67b/escrow_protocol_ico_crowdfunding_platform_with/h81jxlg/,ESCROW PROTOCOL are aiming to attract users from outside of the cryptocurrency space
4,Gabus_Bego,t2_bbs0wmdt,False,"Thank you, my dude.",NaN,NaN,1628333276,1,h81jxk6,t1_h81jtfb,/r/CryptoCurrency/comments/ozinnf/daily_discussion_august_7_2021_gmt0/h81jxk6/,Thank you my dude


In [135]:
# Tokenize words
import re

# Function to Tokenize words
def tokenize(text):
    tokens = re.split('\W+', text) #W+ means that either a word character (A-Za-z0-9_) or a dash (-) can go there.
    return tokens

df['body_text_tokenized'] = df['body_text_clean'].apply(lambda x: tokenize(x.lower())) 
#We convert to lower as Python is case-sensitive. 

df.head()

,author,author_fullname,author_premium,body,collapsed_reason_code,comment_type,created_utc,score,id,parent_id,permalink,body_text_clean,body_text_tokenized
0,HiCarumba,t2_95jx76vi,False,Yes Brother. This is the way.,NaN,NaN,1628333279,1,h81jxny,t1_h81jvaa,/r/CryptoCurrency/comments/ozr28z/it_was_prophesied/h81jxny/,Yes Brother This is the way,"[yes, brother, this, is, the, way]"
1,baeiby,t2_7htt0be3,False,"Lol to be very honest, I knew close to zilch abt investing in general. Didn't even know what DCA...",NaN,NaN,1628333279,2,h81jxnr,t1_h81jn5u,/r/CryptoCurrency/comments/ozqz9m/i_trust_people_here_more_than_my_circle_of/h81jxnr/,Lol to be very honest I knew close to zilch abt investing in general Didnt even know what DCA me...,"[lol, to, be, very, honest, i, knew, close, to, zilch, abt, investing, in, general, didnt, even,..."
2,NotRyanPace,t2_xgk0k,False,"End of year, take the ""Last call for ETH under $10k"" comments seriously. That's a reasonable pri...",NaN,NaN,1628333278,1,h81jxmq,t1_h81jqki,/r/CryptoCurrency/comments/ozinnf/daily_discussion_august_7_2021_gmt0/h81jxmq/,End of year take the Last call for ETH under 10k comments seriously Thats a reasonable price EOY,"[end, of, year, take, the, last, call, for, eth, under, 10k, comments, seriously, thats, a, reas..."
3,BountyManagerBD,t2_8nbdzghw,False,ESCROW PROTOCOL are aiming to attract users from outside of the cryptocurrency space,NaN,NaN,1628333277,1,h81jxlg,t3_o2e67b,/r/CryptoCurrency/comments/o2e67b/escrow_protocol_ico_crowdfunding_platform_with/h81jxlg/,ESCROW PROTOCOL are aiming to attract users from outside of the cryptocurrency space,"[escrow, protocol, are, aiming, to, attract, users, from, outside, of, the, cryptocurrency, space]"
4,Gabus_Bego,t2_bbs0wmdt,False,"Thank you, my dude.",NaN,NaN,1628333276,1,h81jxk6,t1_h81jtfb,/r/CryptoCurrency/comments/ozinnf/daily_discussion_august_7_2021_gmt0/h81jxk6/,Thank you my dude,"[thank, you, my, dude]"


In [136]:
# Remove stopwords

import nltk

stopword = nltk.corpus.stopwords.words('english')# All English Stopwords

In [137]:
# Function to remove Stopwords
def remove_stopwords(tokenized_list):
    text = [word for word in tokenized_list if word not in stopword]# To remove all stopwords
    return text

df['body_text_nostop'] = df['body_text_tokenized'].apply(lambda x: remove_stopwords(x))

df.head()

,author,author_fullname,author_premium,body,collapsed_reason_code,comment_type,created_utc,score,id,parent_id,permalink,body_text_clean,body_text_tokenized,body_text_nostop
0,HiCarumba,t2_95jx76vi,False,Yes Brother. This is the way.,NaN,NaN,1628333279,1,h81jxny,t1_h81jvaa,/r/CryptoCurrency/comments/ozr28z/it_was_prophesied/h81jxny/,Yes Brother This is the way,"[yes, brother, this, is, the, way]","[yes, brother, way]"
1,baeiby,t2_7htt0be3,False,"Lol to be very honest, I knew close to zilch abt investing in general. Didn't even know what DCA...",NaN,NaN,1628333279,2,h81jxnr,t1_h81jn5u,/r/CryptoCurrency/comments/ozqz9m/i_trust_people_here_more_than_my_circle_of/h81jxnr/,Lol to be very honest I knew close to zilch abt investing in general Didnt even know what DCA me...,"[lol, to, be, very, honest, i, knew, close, to, zilch, abt, investing, in, general, didnt, even,...","[lol, honest, knew, close, zilch, abt, investing, general, didnt, even, know, dca, meant, perhap..."
2,NotRyanPace,t2_xgk0k,False,"End of year, take the ""Last call for ETH under $10k"" comments seriously. That's a reasonable pri...",NaN,NaN,1628333278,1,h81jxmq,t1_h81jqki,/r/CryptoCurrency/comments/ozinnf/daily_discussion_august_7_2021_gmt0/h81jxmq/,End of year take the Last call for ETH under 10k comments seriously Thats a reasonable price EOY,"[end, of, year, take, the, last, call, for, eth, under, 10k, comments, seriously, thats, a, reas...","[end, year, take, last, call, eth, 10k, comments, seriously, thats, reasonable, price, eoy]"
3,BountyManagerBD,t2_8nbdzghw,False,ESCROW PROTOCOL are aiming to attract users from outside of the cryptocurrency space,NaN,NaN,1628333277,1,h81jxlg,t3_o2e67b,/r/CryptoCurrency/comments/o2e67b/escrow_protocol_ico_crowdfunding_platform_with/h81jxlg/,ESCROW PROTOCOL are aiming to attract users from outside of the cryptocurrency space,"[escrow, protocol, are, aiming, to, attract, users, from, outside, of, the, cryptocurrency, space]","[escrow, protocol, aiming, attract, users, outside, cryptocurrency, space]"
4,Gabus_Bego,t2_bbs0wmdt,False,"Thank you, my dude.",NaN,NaN,1628333276,1,h81jxk6,t1_h81jtfb,/r/CryptoCurrency/comments/ozinnf/daily_discussion_august_7_2021_gmt0/h81jxk6/,Thank you my dude,"[thank, you, my, dude]","[thank, dude]"


In [138]:
# Preprocessing data

ps = nltk.PorterStemmer()

def stemming(tokenized_text):
    text = [ps.stem(word) for word in tokenized_text]
    return text

df['body_text_stemmed'] = df['body_text_nostop'].apply(lambda x: stemming(x))

df.head()

,author,author_fullname,author_premium,body,collapsed_reason_code,comment_type,created_utc,score,id,parent_id,permalink,body_text_clean,body_text_tokenized,body_text_nostop,body_text_stemmed
0,HiCarumba,t2_95jx76vi,False,Yes Brother. This is the way.,NaN,NaN,1628333279,1,h81jxny,t1_h81jvaa,/r/CryptoCurrency/comments/ozr28z/it_was_prophesied/h81jxny/,Yes Brother This is the way,"[yes, brother, this, is, the, way]","[yes, brother, way]","[ye, brother, way]"
1,baeiby,t2_7htt0be3,False,"Lol to be very honest, I knew close to zilch abt investing in general. Didn't even know what DCA...",NaN,NaN,1628333279,2,h81jxnr,t1_h81jn5u,/r/CryptoCurrency/comments/ozqz9m/i_trust_people_here_more_than_my_circle_of/h81jxnr/,Lol to be very honest I knew close to zilch abt investing in general Didnt even know what DCA me...,"[lol, to, be, very, honest, i, knew, close, to, zilch, abt, investing, in, general, didnt, even,...","[lol, honest, knew, close, zilch, abt, investing, general, didnt, even, know, dca, meant, perhap...","[lol, honest, knew, close, zilch, abt, invest, gener, didnt, even, know, dca, meant, perhap, ela..."
2,NotRyanPace,t2_xgk0k,False,"End of year, take the ""Last call for ETH under $10k"" comments seriously. That's a reasonable pri...",NaN,NaN,1628333278,1,h81jxmq,t1_h81jqki,/r/CryptoCurrency/comments/ozinnf/daily_discussion_august_7_2021_gmt0/h81jxmq/,End of year take the Last call for ETH under 10k comments seriously Thats a reasonable price EOY,"[end, of, year, take, the, last, call, for, eth, under, 10k, comments, seriously, thats, a, reas...","[end, year, take, last, call, eth, 10k, comments, seriously, thats, reasonable, price, eoy]","[end, year, take, last, call, eth, 10k, comment, serious, that, reason, price, eoy]"
3,BountyManagerBD,t2_8nbdzghw,False,ESCROW PROTOCOL are aiming to attract users from outside of the cryptocurrency space,NaN,NaN,1628333277,1,h81jxlg,t3_o2e67b,/r/CryptoCurrency/comments/o2e67b/escrow_protocol_ico_crowdfunding_platform_with/h81jxlg/,ESCROW PROTOCOL are aiming to attract users from outside of the cryptocurrency space,"[escrow, protocol, are, aiming, to, attract, users, from, outside, of, the, cryptocurrency, space]","[escrow, protocol, aiming, attract, users, outside, cryptocurrency, space]","[escrow, protocol, aim, attract, user, outsid, cryptocurr, space]"
4,Gabus_Bego,t2_bbs0wmdt,False,"Thank you, my dude.",NaN,NaN,1628333276,1,h81jxk6,t1_h81jtfb,/r/CryptoCurrency/comments/ozinnf/daily_discussion_august_7_2021_gmt0/h81jxk6/,Thank you my dude,"[thank, you, my, dude]","[thank, dude]","[thank, dude]"


In [49]:
# Preprocessing Data with Lemmatizer

wn = nltk.WordNetLemmatizer()

def lemmatizing(tokenized_text):
    text = [wn.lemmatize(word) for word in tokenized_text]
    return text

df['body_text_lemmatized'] = df['body_text_nostop'].apply(lambda x: lemmatizing(x))

df.head(10)

,author,author_fullname,author_premium,body,collapsed_reason_code,comment_type,created_utc,score,id,parent_id,...,polarity,neg,neu,pos,compound,body_text_clean,body_text_tokenized,body_text_nostop,body_text_stemmed,body_text_lemmatized
0,HiCarumba,t2_95jx76vi,False,0 Yes Brother. This...,NaN,NaN,1628333279,1,h81jxny,t1_h81jvaa,...,"{'neg': 0.0, 'neu': 0.784, 'pos': 0.216, 'comp...",0.0,0.784,0.216,0.9705,0 Yes Brother This ...,"[0, yes, brother, this, is, the, way, 1, lol, ...","[0, yes, brother, way, 1, lol, honest, knew, c...","[0, ye, brother, way, 1, lol, honest, knew, cl...","[0, yes, brother, way, 1, lol, honest, knew, c..."
1,baeiby,t2_7htt0be3,False,0 Yes Brother. This...,NaN,NaN,1628333279,2,h81jxnr,t1_h81jn5u,...,"{'neg': 0.0, 'neu': 0.784, 'pos': 0.216, 'comp...",0.0,0.784,0.216,0.9705,0 Yes Brother This ...,"[0, yes, brother, this, is, the, way, 1, lol, ...","[0, yes, brother, way, 1, lol, honest, knew, c...","[0, ye, brother, way, 1, lol, honest, knew, cl...","[0, yes, brother, way, 1, lol, honest, knew, c..."
2,NotRyanPace,t2_xgk0k,False,0 Yes Brother. This...,NaN,NaN,1628333278,1,h81jxmq,t1_h81jqki,...,"{'neg': 0.0, 'neu': 0.784, 'pos': 0.216, 'comp...",0.0,0.784,0.216,0.9705,0 Yes Brother This ...,"[0, yes, brother, this, is, the, way, 1, lol, ...","[0, yes, brother, way, 1, lol, honest, knew, c...","[0, ye, brother, way, 1, lol, honest, knew, cl...","[0, yes, brother, way, 1, lol, honest, knew, c..."
3,BountyManagerBD,t2_8nbdzghw,False,0 Yes Brother. This...,NaN,NaN,1628333277,1,h81jxlg,t3_o2e67b,...,"{'neg': 0.0, 'neu': 0.784, 'pos': 0.216, 'comp...",0.0,0.784,0.216,0.9705,0 Yes Brother This ...,"[0, yes, brother, this, is, the, way, 1, lol, ...","[0, yes, brother, way, 1, lol, honest, knew, c...","[0, ye, brother, way, 1, lol, honest, knew, cl...","[0, yes, brother, way, 1, lol, honest, knew, c..."
4,Gabus_Bego,t2_bbs0wmdt,False,0 Yes Brother. This...,NaN,NaN,1628333276,1,h81jxk6,t1_h81jtfb,...,"{'neg': 0.0, 'neu': 0.784, 'pos': 0.216, 'comp...",0.0,0.784,0.216,0.9705,0 Yes Brother This ...,"[0, yes, brother, this, is, the, way, 1, lol, ...","[0, yes, brother, way, 1, lol, honest, knew, c...","[0, ye, brother, way, 1, lol, honest, knew, cl...","[0, yes, brother, way, 1, lol, honest, knew, c..."
5,Tarskin_Tarscales,t2_3jjabb6f,False,0 Yes Brother. This...,NaN,NaN,1628333276,1,h81jxk4,t3_ozqfrr,...,"{'neg': 0.0, 'neu': 0.784, 'pos': 0.216, 'comp...",0.0,0.784,0.216,0.9705,0 Yes Brother This ...,"[0, yes, brother, this, is, the, way, 1, lol, ...","[0, yes, brother, way, 1, lol, honest, knew, c...","[0, ye, brother, way, 1, lol, honest, knew, cl...","[0, yes, brother, way, 1, lol, honest, knew, c..."
6,atomwest314,t2_x50okt,True,0 Yes Brother. This...,NaN,NaN,1628333275,3,h81jxhz,t3_ozr2hx,...,"{'neg': 0.0, 'neu': 0.784, 'pos': 0.216, 'comp...",0.0,0.784,0.216,0.9705,0 Yes Brother This ...,"[0, yes, brother, this, is, the, way, 1, lol, ...","[0, yes, brother, way, 1, lol, honest, knew, c...","[0, ye, brother, way, 1, lol, honest, knew, cl...","[0, yes, brother, way, 1, lol, honest, knew, c..."
7,Chance_Interaction35,t2_7hi4xpjr,False,0 Yes Brother. This...,NaN,NaN,1628333274,2,h81jxgs,t1_h81jv1j,...,"{'neg': 0.0, 'neu': 0.784, 'pos': 0.216, 'comp...",0.0,0.784,0.216,0.9705,0 Yes Brother This ...,"[0, yes, brother, this, is, the, way, 1, lol, ...","[0, yes, brother, way, 1, lol, honest, knew, c...","[0, ye, brother, way, 1, lol, honest, knew, cl...","[0, yes, brother, way, 1, lol, honest, knew, c..."
8,MrNuttyJoe,t2_9yo6aez0,True,0 Yes Brother. This...,NaN,NaN,1628333273,5,h81jxg8,t1_h81jd7s,...,"{'neg': 0.0, 'neu': 0.784, 'pos': 0.216, 'comp...",0.0,0.784,0.216,0.9705,0 Yes Brother This ...,"[0, yes, brother, this, is, the, way, 1, lol, ...","[0, yes, brother, way, 1, lol, honest, knew, c...","[0, ye, brother, way, 1, lol, honest, knew, cl...","[0, yes, brother, way, 1, lol, honest, knew, c..."
9,Professional_Crow485,t2_b6zo83z2,False,0 Yes Brother. This...,NaN,NaN,1628333273,2,h81jxg6,t1_h81jsox,...,"{'neg': 0.

In [139]:
# Saving cleaned dataframe
df.to_csv("../Data/Reddit_Comments/Cleaned/comments_cleaned.csv")

## Vectorize Data using Bag of Words

In [57]:
import pandas as pd
import re
import string
import nltk
pd.set_option('display.max_colwidth', 100) # To extend column width

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

comments_df.columns = ['label', 'body']

ValueError: Length mismatch: Expected axis has 25 elements, new values have 2 elements